# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [80]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data
import numpy as np

training_file = 'train.p'
validation_file= 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']
print('done loading')

done loading


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [81]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(X_train)
#y = np.bincount(y_valid)
#ii = np.nonzero(y)[0]
#print(np.vstack((ii,y[ii])).T)
# TODO: Number of testing examples.
n_test = len(X_test)


# TODO: Number of testing examples.
n_valid = len(X_valid)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape
#sizes = X_train[0]#sizes,X_train[0].coords

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_valid))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Number of validation examples =", n_valid)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)


Number of training examples = 34799
Number of testing examples = 12630
Number of validation examples = 4410
Image data shape = (32, 32, 3)
Number of classes = 43


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [82]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

In [83]:
### Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

import cv2
import random
import time
from sklearn.utils import shuffle

In [84]:
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)
#this with tepical LeNet gives  .86 ~ .89  
def preprocess_image_set(data, label,is_training_data=True):
    labels_with_small_data = [0,6,15,16,19,20,21,22,23,24,27,28,29,30,32,34,36,37,39,40,41,42]
    process_time = time.time()
    #new_data = np.empty_like (data)
    new_data = np.zeros(data.shape[0]*data.shape[1]*data.shape[2])
    new_data=new_data.reshape(data.shape[0],data.shape[1],data.shape[2],1)
    print("new_data.shape",new_data.shape)
    new_data = new_data.tolist()
    label = label.tolist()
    for i in range(len(data)):
        img = data[i]
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        cv2.normalize(img,img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)#, dtype=cv2.CV_32F)
        if(is_training_data):
            apply_effects = False
            try : 
                labels_with_small_data.index(label[i])
                apply_effects = True
            except :
                apply_effects = False

            if apply_effects:
                img = cv2.equalizeHist(img)
                #img = tf.image.random_brightness(img, max_delta=.6)
                #img = tf.image.random_contrast(img, lower=3, upper=5)
                #img = tf.image.adjust_saturation(img, 0)
                #img = tf.image.per_image_standardization(img)
                random_val = random.triangular(.5, 1.1)#best random value
                img = adjust_gamma(img,random_val)#1.17)
                #random_val = random.randint(-1,1) * 90 
                #translation deccreses accuracy ~ 2%
                #cv2.getRotationMatrix2D decreases accuracy ~ 1%
                rows,cols = img.shape
                M = cv2.getRotationMatrix2D((cols/2,rows/2),20,1)
                img1 = cv2.warpAffine(img,M,(cols,rows))
                img1.shape = (img.shape[0],img.shape[1],1)
                new_data.append(img1)
                label.append(label[i])
                M = cv2.getRotationMatrix2D((cols/2,rows/2),-20,1)
                img2 = cv2.warpAffine(img,M,(cols,rows))
                img2.shape = (img.shape[0],img.shape[1],1)
                new_data.append(img2)
                label.append(label[i])
        img.shape = (img.shape[0],img.shape[1],1)
        new_data[i] = img
    process_time = time.time() - process_time
    print("process_time",process_time)
    #return new_data , label
    return np.array(new_data) , np.array(label)

X_train_new, y_train_new = X_train, y_train#X_train[0:int(n_train*.1)], y_train[0:int(n_train*.1)]#
X_test_tst, y_test_tst = X_test, y_test#X_test[0:int(n_test*.1)], y_test[0:int(n_test*.001)]#
X_valid_tst, y_valid_tst = X_valid, y_valid#X_valid[0:int(n_valid*.1)], y_valid[0:int(n_valid*.1)]#

print("before- training examples =", len(X_train))
print("before- training labels =", len(y_train))

X_train_new,y_train_new=preprocess_image_set(X_train_new,y_train_new)
X_valid_tst,y_valid_tst=preprocess_image_set(X_valid_tst,y_valid_tst,is_training_data=False)
X_test_tst,y_test_tst=preprocess_image_set(X_test_tst,y_test_tst,is_training_data=False)

print("after - training examples =", len(X_train_new))
print("after - training labels =", len(y_train_new))




before- training examples = 34799
before- training labels = 34799
new_data.shape (34799, 32, 32, 1)
process_time 38.69773244857788
new_data.shape (4410, 32, 32, 1)
process_time 2.4684765338897705
new_data.shape (12630, 32, 32, 1)
process_time 11.173027515411377
after - training examples = 47999
after - training labels = 47999


In [85]:
#try with 20% of train data

X_train_tst, y_train_tst = shuffle(X_train_new, X_train_new)
#X_train_tst, y_train_tst = X_train_tst[0:int(len(X_train_tst)*.5)], y_train_tst[0:int(len(y_train_tst)*.5)]#

print("20% - training examples =", len(X_train_tst))
print("20% - training labels =", len(y_train_tst))


"""
#rand_index = 20024 #random.randint(0, int(len(X_train) -1))# 275 #
print("index",rand_index,"of",y_train_tst[rand_index])
img0 = X_train_tst[rand_index]#.squeeze()
plt.figure()
plt.imshow(img0.squeeze())
#import sys
#sys.exit()

#print(index,"label",y_train[index])
img0 = X_train_tst[rand_index]#.squeeze()
plt.figure()
plt.imshow(img0.squeeze(), cmap='gray')
"""

20% - training examples = 47999
20% - training labels = 47999


'\n#rand_index = 20024 #random.randint(0, int(len(X_train) -1))# 275 #\nprint("index",rand_index,"of",y_train_tst[rand_index])\nimg0 = X_train_tst[rand_index]#.squeeze()\nplt.figure()\nplt.imshow(img0.squeeze())\n#import sys\n#sys.exit()\n\n#print(index,"label",y_train[index])\nimg0 = X_train_tst[rand_index]#.squeeze()\nplt.figure()\nplt.imshow(img0.squeeze(), cmap=\'gray\')\n'

## Features and Labels

In [86]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf
from tensorflow.contrib.layers import flatten

In [87]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

### Model Architecture

In [88]:
keep_prob = tf.placeholder(tf.float32)
EPOCHS = 50
BATCH_SIZE = 128
SAME = 'SAME'
VALID = 'VALID'

In [89]:
def conv2d(x, W, b, strides=1,padding='SAME'):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=padding)
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [90]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [91]:
#def LeNet(x):    
# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
l2_scale = .01#.001
mu = 0
sigma = .1
weights = {
    #random_normal#truncated_normal
'wc1': tf.Variable(tf.random_normal([5, 5, 1, 6], mean = mu, stddev = sigma)),
'wc2': tf.Variable(tf.random_normal([5, 5, 6, 16], mean = mu, stddev = sigma)),
'wd1': tf.Variable(tf.random_normal([400, 120], mean = mu, stddev = sigma)),
'wd2': tf.Variable(tf.random_normal([120, 84], mean = mu, stddev = sigma)),
'wd3': tf.Variable(tf.random_normal([84, n_classes], mean = mu, stddev = sigma))}
biases = {
'bc1': tf.Variable(tf.zeros([6])),
'bc2': tf.Variable(tf.zeros([16])),
'bd1': tf.Variable(tf.zeros([120])),
'bd2': tf.Variable(tf.zeros([84])),
'bd3': tf.Variable(tf.zeros([n_classes]))}
"""
regularizer = tf.nn.l2_loss(weights)
loss = tf.reduce_mean(loss + beta * regularizer)
"""
def sum_of_all_l2_loss():
    sum_of_l2_loss = (tf.nn.l2_loss(weights['wc1']) + 
        #tf.nn.l2_loss(biases['bc1']) + 
        tf.nn.l2_loss(weights['wc2']) + 
        #tf.nn.l2_loss(biases['bc2']) + 
        tf.nn.l2_loss(weights['wd1']) + 
        #tf.nn.l2_loss(biases['bd1']) + 
        tf.nn.l2_loss(weights['wd2']) + 
        #tf.nn.l2_loss(biases['bd2']) + 
        tf.nn.l2_loss(weights['wd3'])) 
        #tf.nn.l2_loss(biases['bd3']))
    return l2_scale*sum_of_l2_loss
# TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.# TODO: Activation.
#Using VALID 32-5(from weights)+1

conv1 = conv2d(x, weights['wc1'], biases['bc1'],padding=VALID)
conv1 = tf.contrib.layers.batch_norm(conv1, 
                                  center=True, scale=True, 
                                  is_training=True)
#print(conv1.shape)#(?, 28, 28, 6)
# TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
conv1 = maxpool2d(conv1, k=2)
#print(conv1.shape)#(?, 14, 14, 6)

# TODO: Layer 2: Convolutional. Output = 10x10x16.# TODO: Activation.
#Using VALID 14-5(from weights)+1
conv2 = conv2d(conv1, weights['wc2'], biases['bc2'],padding=VALID)
conv2 = tf.contrib.layers.batch_norm(conv2, 
                                  center=True, scale=True, 
                                  is_training=True)
#print(conv2.shape)#(?, 10, 10, 16)
# TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
conv2 = maxpool2d(conv2, k=2)
#print(conv2.shape)#(?, 5, 5, 16)

# TODO: Flatten. Input = 5x5x16. Output = 400.
flattend = flatten(inputs=conv2)
#print(flattend.shape)#400
# TODO: Layer 3: Fully Connected. Input = 400. Output = 120.# TODO: Activation.
fc1 = tf.add(tf.matmul(flattend, weights['wd1']), biases['bd1'])
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, keep_prob)
#fc1 = tf.nn.dropout(fc1, dropout)
#print(fc1.shape)#400
# TODO: Layer 4: Fully Connected. Input = 120. Output = 84.#TODO: Activation.
fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
fc2 = tf.nn.relu(fc2)
fc2 = tf.nn.dropout(fc2, keep_prob)

#print(fc2.shape)#84
# TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
logits = tf.add(tf.matmul(fc2, weights['wd3']), biases['bd3'])
#print(logits.shape)#84
#    return logits


## Training Pipeline

In [92]:
rate = 0.001#
rate = 0.01#
rate = 0.0008

#logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)# + sum_of_all_l2_loss())
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

## Model Evaluation

In [93]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train, Validate and Test the Model
A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [94]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #num_examples = len(X_train)
    num_examples = len(X_train_tst)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        #X_train, y_train = shuffle(X_train, y_train)
        X_train_tst, y_train_tst = shuffle(X_train_tst, y_train_tst)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_tst[offset:end], y_train_tst[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.1})
            
        validation_accuracy = evaluate(X_valid_tst, y_valid_tst)
        print("EPOCH {} ... ".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...



InvalidArgumentError: logits and labels must be same size: logits_size=[128,43] labels_size=[131072,43]
	 [[Node: SoftmaxCrossEntropyWithLogits_17 = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape_51, Reshape_52)]]

Caused by op 'SoftmaxCrossEntropyWithLogits_17', defined at:
  File "C:\Users\Omnia\Anaconda3\envs\carnd-term1\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Omnia\Anaconda3\envs\carnd-term1\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Omnia\Anaconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\Omnia\Anaconda3\envs\carnd-term1\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Omnia\Anaconda3\envs\carnd-term1\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().sta

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")
            
outputFeatureMap(X_train_tst[rand_index],conv1)

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 